In [ ]:
%pip install optimum
%pip install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
%pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu117/  # Use cu118 if on CUDA 11.8


In [ ]:
# @title Load the model: Mistral7B-instruct-v0.1
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name_or_path = 'TheBloke/Mistral-7B-Instruct-v0.1-GPTQ'
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision='main')

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
streamer = TextStreamer(tokenizer, skip_prompt=True)



In [ ]:
# @title Ask it anything
prompt = 'Schrijf een gedicht over incompetente managers die zich verschuilen achter moeders rok. In hey Nederlands'
prompt_template=f'''<s>[INST] {prompt} [/INST]
'''

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()


output = model.generate(input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=2048, repetition_penalty=1.1, streamer=streamer)